In [151]:
#baseline
from sklearn.linear_model import Lasso

In [152]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

In [153]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [154]:
X = pd.read_pickle('../data/pkl/X_preproc_rolling_advanced_part1.pkl')

In [155]:
X= X.drop(labels=['HOME_TEAM_h','HOME_TEAM_a'], axis=1)

In [156]:
y = pd.read_pickle('../data/pkl/y_preproc_rolling_advanced_part1.pkl')

In [157]:
X.columns = X.columns.astype(str)

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)

In [159]:
y_true_mean = y.mean()

In [160]:
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import cross_val_score
solver = "highs"
regression = QuantileRegressor(quantile=0.5, alpha=0.0065, solver=solver)
y_pred = regression.fit(X_train, y_train).predict(X_test)
np.mean(cross_val_score(regression, X_train, y_train, cv=5, scoring='r2'))

0.3940688427810052

In [161]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
y_pred = lin_reg.fit(X_train, y_train).predict(X_test)
np.mean(cross_val_score(lin_reg, X_train, y_train, cv=5, scoring='r2'))

0.36425109078190776

In [38]:
!pip install quantile-forest

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 678.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 5.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: cython
    Found existing installation: Cython 0.29.32
    Uninstalling Cython-0.29.32:
      Successfully uninstalled Cython-0.29.32


In [162]:
from quantile_forest import RandomForestQuantileRegressor
rfqr = RandomForestQuantileRegressor(n_estimators=700)
#rfqr.fit(X_train, y_train)
#predictions = rfqr.predict(X_test, quantiles=[0.025, 0.5, 0.975]) 

In [163]:
param_grid = {'bootstrap': [True, False],
 'max_depth': randint(10,100),
 'min_samples_leaf': randint(1,8),
 'min_samples_split': randint(1,10),
  'max_features': ['auto', 'sqrt', 'log2']}

In [164]:
rf_random = RandomizedSearchCV(estimator = rfqr, param_distributions = param_grid)

In [142]:
rf_random.fit(X_train, y_train)


KeyboardInterrupt



In [131]:
rf_random.best_params_

{'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [129]:
best_random = rf_random.best_estimator_
rcv_predictions = best_random.predict(X_test, quantiles=[0.025, 0.5, 0.975])

In [130]:
r2_score(y_test, rcv_predictions[:,1])

0.39122603588570304

In [121]:
from sklearn.metrics import r2_score
r2_score(y_test, predictions[:,1])

0.4105454018809318

In [134]:
from sklearn.base import clone

In [165]:
grid_search = RandomizedSearchCV(rfqr, param_distributions=param_grid,
                                    n_iter=30, cv=5, random_state=123,
                                   verbose=1, n_jobs=-1, #refit='neg_mean_squared_error',
                                   scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)# ,**fit_params
print(grid_search.best_estimator_)
best_params = grid_search.best_estimator_.get_params()
#     return grid
alg= clone(rfqr).set_params(**best_params)
alg=alg.fit(X_train, y_train)#, **fit_params
pred=alg.predict(X_test)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
RandomForestQuantileRegressor(max_depth=36, max_features='auto',
                              min_samples_leaf=7, min_samples_split=5,
                              n_estimators=700)


In [166]:
r2_score(y_test, pred)

0.4189268123059331

In [82]:
df_preds = pd.DataFrame(predictions, columns={'0.025_quantile', '0.5_quantile', '0.975_quantile'})

In [74]:
visualise_cols = ['HOME_TEAM_h', 'OFF_RATING_h', 'DEF_RATING_h', 'NET_RATING_h',
       'AST_PCT_h', 'AST_TOV_h', 'OREB_PCT_h', 'DREB_PCT_h', 'REB_PCT_h',
       'TM_TOV_PCT_h', 'EFG_PCT_h', 'TS_PCT_h', 'PACE_h', 'POSS_h',
       'HOME_TEAM_a', 'OFF_RATING_a', 'DEF_RATING_a', 'NET_RATING_a',
       'AST_PCT_a', 'AST_TOV_a', 'OREB_PCT_a', 'DREB_PCT_a', 'REB_PCT_a',
       'TM_TOV_PCT_a', 'EFG_PCT_a', 'TS_PCT_a', 'PACE_a', 'POSS_a', 'ATL_h']

In [93]:
df_merge = pd.concat([y_test, df_preds], axis=1)

In [102]:
y_test.reset_index()

,index,PLUS_MINUS
0,923,18.0
1,440,-9.0
2,459,16.0
3,2207,-33.0
4,53,10.0
...,...,...
454,317,3.0
455,1544,-25.0
456,1236,18.0
457,1226,-1.0


In [103]:
df_preds['true'] = y_test.reset_index()['PLUS_MINUS']

In [104]:
df_preds

,0.025_quantile,0.5_quantile,0.975_quantile,true
0,-32.000,3.0,33.000,18.0
1,-21.000,-1.0,18.525,-9.0
2,-7.525,11.0,39.100,16.0
3,-22.050,-8.0,11.575,-33.0
4,-20.050,1.0,21.000,10.0
...,...,...,...,...
454,-3.050,11.0,45.000,3.0
455,-38.000,-18.0,6.000,-25.0
456,-19.000,12.0,43.000,18.0
457,-22.050,-2.0,22.050,-1.0


In [92]:
plt.figure(figsize=[10,6])
sns.pairplot(data=df_merge, x_vars=visualise_cols, y_vars=['PLUS_MINUS', '0.025_quantile', '0.5_quantile', '0.975_quantile'], height=4, aspect=4)
plt.show()

<Figure size 2000x800 with 0 Axes>

In [56]:
np.array(y_test).shape

(459,)

In [62]:
import quantile_scoring as scr
alpha_test= 0.2
scr.interval_score(np.array(y_test),alpha_test,q_left=predictions[:,0],q_right=predictions[:,-1])

(array([ 55.1  ,  38.715,  42.1  , 154.8  ,  34.725,  35.525,  50.2  ,
         31.   ,  42.625,  44.2  ,  62.2  ,  82.1  ,  35.625,  48.375,
         44.05 ,  40.   ,  52.25 ,  36.525,  46.1  ,  40.2  ,  39.525,
        130.15 ,  41.   ,  44.625,  48.525,  29.   ,  37.775,  35.05 ,
         42.575,  40.68 ,  43.05 ,  44.05 ,  39.775,  36.   ,  41.05 ,
         38.525,  34.575,  41.   ,  41.625,  42.625,  32.575,  55.525,
         43.05 ,  46.575,  30.575,  49.15 ,  41.05 ,  46.   ,  36.05 ,
         41.1  ,  34.525,  46.1  ,  42.05 ,  39.05 ,  62.3  ,  37.525,
         46.25 ,  44.525,  48.1  ,  44.55 ,  53.   ,  45.05 ,  38.525,
         39.05 ,  66.   ,  51.   ,  93.125,  27.05 ,  39.05 ,  35.525,
         49.   ,  37.575,  40.15 ,  43.05 ,  74.   ,  34.05 ,  42.   ,
         41.625,  34.525,  47.575,  36.2  ,  33.05 ,  34.05 ,  45.525,
         47.725,  44.1  ,  56.   , 154.485,  37.625,  32.15 ,  36.575,
         37.525,  40.   ,  37.4  ,  37.575,  42.625,  31.1  ,  38.625,
      

In [335]:
X_2 = df_merged.drop(['PTS', 'TEAM_ID', 'OFF_RATING', 'EFG_PCT'], axis=1)
y_2 = df_merged['PTS']

In [336]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X_2, y_2, random_state=0, test_size=0.2)

In [337]:
mx_scaler2 = MinMaxScaler()
X2_train = mx_scaler.fit_transform(X2_train)
X2_test = mx_scaler.transform(X2_test)

In [338]:
y2_true_mean = y_2.mean()

In [345]:
predictions2 = {}
scores2 = {}
out_bounds_predictions = np.zeros_like(y2_true_mean, dtype=np.bool_)
for quantile in quantiles:
    qr2 = QuantileRegressor(quantile=quantile, alpha=0.005, solver=solver)
    y2_pred = qr.fit(X2_train, y2_train).predict(X2_test)
    predictions2[quantile] = y2_pred
    scores2[quantile] = cross_val_score(qr2, X2_train, y2_train, cv=5, scoring='r2')

    if quantile == min(quantiles):
        out_bounds_predictions = np.logical_or(
            out_bounds_predictions, y2_pred >= y2_test
        )
    elif quantile == max(quantiles):
        out_bounds_predictions = np.logical_or(
            out_bounds_predictions, y2_pred <= y2_test
        )

In [346]:
print(scores2[0.025])
print(scores2[0.5])
print(scores2[0.975])

[-3.28337394 -3.79601085 -3.50640535 -3.37807261 -3.08342673]
[0.99762027 0.99778954 0.99588966 0.99770141 0.99662594]
[-3.61416404 -3.66402199 -3.81771701 -5.11401248 -3.98919511]
